In [4]:
nprocs()

1

In [1]:
1+1

2

In [3]:
using Distributed, LinearAlgebra
#rmprocs(workers())


# A Brief Feel for the Dream

In [5]:
import Pkg; Pkg.add("CUDA")

    Updating registry at `~/data/.julia/registries/JuliaComputingRegistry.toml`


   Resolving package versions...


  No Changes to `/mnt/data/.julia/environments/v1.7/Project.toml`
  No Changes to `/mnt/data/.julia/environments/v1.7/Manifest.toml`


In [6]:
using Distributed

if nprocs() == 1
  addprocs(4) 
end

# @everywhere begin
#     import Pkg
#     Pkg.activate($(Base.current_project()))
#   end

@everywhere using LinearAlgebra, DistributedArrays, CUDA
@everywhere import CUDA: @time
@everywhere CUDA.allowscalar(true)


## So you like linear algebra? <br>
<table style="width:100%">
  <tr>
    <th><img src="http://freecomputerbooks.com/covers/LAPACK-Users-Guide-Third-Edition.jpg" width="200"></th>
    <th><img src="http://www.netlib.org/scalapack/slug/slug_cover.gif"  width="200" style="background-color:white"></th>
      <th><img src="https://images-na.ssl-images-amazon.com/images/I/41uT31fSxBL._SX352_BO1,204,203,200_.jpg" width="200"></th>
    
  </tr>
    



## but you are innovating beyond the traditional libraries?

##  For demonstration let's make a matrix where you know more than the libraries know.

Let's create a custom  matrix structure that depends on n parameters, not $n^2$: <br> <br>
Diagonal(v) + v*v'
<br>

For example (n=3):
$$ \begin{pmatrix}
v_1 & & \\
& v_2 & \\
& & v_3
\end{pmatrix} + 
\begin{pmatrix}
v_1^2 & v_1 v_2 & v_1 v_3 \\
v_2 v_1 & v_2^2 & v_2 v_3 \\
v_3 v_1 & v_3 v_2 & v_3^2
\end{pmatrix} .
$$


# Build a Custom Type

In [7]:
struct MyMatrix{T, V<:AbstractVector{T}} <: AbstractMatrix{T}
   v::V
end

In [8]:
Base.size(A::MyMatrix)         =   length(A.v), length(A.v)
Base.getindex(A::MyMatrix,i,j) =   A.v[i]*(i==j)  + A.v[i]*A.v[j]


In [9]:
A = MyMatrix([1,10,100])

3×3 MyMatrix{Int64, Vector{Int64}}:
   2    10    100
  10   110   1000
 100  1000  10100

In [10]:
dump( A )  # n storage

MyMatrix{Int64, Vector{Int64}}
  v: Array{

Int64}((3,)) [1, 10, 100]


In [11]:
dump(Matrix(A)) # n² storage

Array{Int64}((3, 3)) 

[2 10 100; 10 110 1000; 100 1000 10100]


## My very own largest eigensolver for my very own matrices

In [12]:
f(A::MyMatrix)  = λ ->  1 + mapreduce((v) -> v^2 / (v - λ)  , +, A.v)
f′(A::MyMatrix) = λ ->      mapreduce((v) -> v^2 / (v - λ)^2, +, A.v)

function LinearAlgebra.eigmax(A::MyMatrix; tol = eps(2.0), debug = false)
    x0 = maximum(A.v) + maximum(A.v)^2
    δ = f(A)(x0)/f′(A)(x0)
    while abs(δ) > x0 * tol               
        x0 -= δ
        δ = f(A)(x0)/f′(A)(x0) 
        debug && println("x = $x0, δ = $δ") # Debugging
    end
    x0
end

In [13]:
eigmax(A)

10200.107083707298

In [14]:
eigmax(Matrix(A))

10200.107083707298

## Go Heterogeneous

In [15]:
gpuA = MyMatrix(CuArray([1,2,3]))

3×3 MyMatrix{Int64, CuArray{Int64, 1, CUDA.Mem.DeviceBuffer}}:
 2  2   3
 2  6   6
 3  6  12

In [16]:
distA = MyMatrix(distribute([1,2,3]))

3×3 MyMatrix{Int64, DArray{Int64, 1, Vector{Int64}}}:
 2  2   3
 2  6   6
 3  6  12

In [17]:
Matrix(gpuA)

3×3 Matrix{Int64}:
 2  2   3
 2  6   6
 3  6  12

In [16]:
Matrix(distA)

3×3 Matrix{Int64}:
 2  2   3
 2  6   6
 3  6  12

# Compare Timings

In [18]:
N = 4_000_000

4000000

In [19]:
v = randn(N)*.1

4000000-element Vector{Float64}:
 -0.053793817582657326
 -0.003592096969330304
 -0.18867427852644192
 -0.01582632633108473
 -0.1832277013847907
  0.005077106926582318
  0.09429267789201608
 -0.09386769921413878
 -0.031091414723704663
  0.12403022622093754
  ⋮
  0.08202188137552724
  0.2054528132742123
  0.1863330435843987
 -0.04871046896917302
 -0.23073206053276693
 -0.10850741820814912
 -0.04602062235592608
 -0.08199493813278634
 -0.21094130703715375

In [22]:
dump(A)

MyMatrix{Float64, Vector{Float64}}
  v: Array{

Float64}((4000000,)) [-0.053793817582657326, -0.003592096969330304, -0.18867427852644192, -0.01582632633108473, -0.1832277013847907, 0.005077106926582318, 0.09429267789201608, -0.09386769921413878, -0.031091414723704663, 0.12403022622093754  …  -0.052266126744157315, 0.08202188137552724, 0.2054528132742123, 0.1863330435843987, -0.04871046896917302, -0.23073206053276693, -0.10850741820814912, -0.04602062235592608, -0.08199493813278634, -0.21094130703715375]


In [21]:
A = MyMatrix(v)

4000000×4000000 MyMatrix{Float64, Vector{Float64}}:
 -0.0509        0.000193233   0.0101495    …   0.00441082    0.0113473
  0.000193233  -0.00357919    0.000677736      0.000294534   0.000757722
  0.0101495     0.000677736  -0.153076         0.0154703     0.0397992
  0.000851359   5.68497e-5    0.00298602       0.00129768    0.00333843
  0.00985652    0.000658172   0.0345704        0.0150237     0.0386503
 -0.000273117  -1.82375e-5   -0.000957919  …  -0.000416297  -0.00107097
 -0.00507236   -0.000338708  -0.0177906       -0.00773152   -0.0198902
  0.0050495     0.000337182   0.0177104        0.00769668    0.0198006
  0.00167253    0.000111683   0.00586615       0.00254934    0.00655846
 -0.00667206   -0.000445529  -0.0234013       -0.0101699    -0.0261631
  ⋮                                        ⋱                
 -0.00441227   -0.000294631  -0.0154754       -0.00672538   -0.0173018
 -0.0110521    -0.000738006  -0.0387637       -0.0168461    -0.0433385
 -0.0100236    -0.000669326  -

In [21]:
@folded_threads eigmax(A) # using a macro

folded_context(true)  # context variable
eigmax(A)

eigmax( Threaded(A) ) # similar to CuArray and distribute (using the type mechanism to figure out to execute)

LoadError: LoadError: UndefVarError: @folded_threads not defined
in expression starting at /home/jrun/data/code/sc.ipynb:1

In [23]:
distA = MyMatrix(distribute(v));  

In [24]:
gpuA = MyMatrix(CuArray(v));

In [24]:
1+1

2

In [25]:
@time eigmax(distA) # run twice

  3.472926 seconds (5.08 M CPU allocations: 

272.595 MiB, 2.29% gc time)


39991.435006823696

In [28]:
CUDA.@timed(eigmax(gpuA)) # run twice

(value = 39991.4350068237, time = 0.004795719, cpu_bytes = 203792, cpu_gctime = 0.0, cpu_gcstats = Base.GC_Diff(203792, 0, 0, 4143, 0, 0, 0, 0, 0), gpu_bytes = 59248, gpu_memtime = 0.0005168400000000739, gpu_memstats = CUDA.AllocStats(92, 59248, 0, 0, 0.0005168400000000739))

In [29]:
@time eigmax(gpuA)

  0.004720 seconds (4.14 k CPU allocations: 199.016 KiB) (92 GPU allocations: 57.859 KiB, 11.30% memmgmt time)


39991.4350068237

## Abstraction

1. A data structure is a mathematical matrix! <br>
2. A matrix can be serial, distributed, gpu <br>
BOTH ARE ABSTRACTIONS, BOTH use underlying similar mechanisms

## Why Julia?

1. Well designed abstractions <br>
2. Multiple dispatch <br>
3. Careful balance between static and dynamic <br>
4. Compiles to GPU at multiple levels, not just lowest CUDA level <br>
5. Metaprogramming across the stack <br>
6. Interfaces with LLVM <br>
7. Plays nicely with Python, legacy codes, ... <br>

<center> <a href="https://arxiv.org/abs/1411.1607">
Julia: A Fresh Approach to Numerical Computing 
</a> </center>

 <center> <img src="siam.png" width=400> </a> </center>